### TO DO:
-Automate the constraint generation

-Correct it, it's finding local mimima

-Find the right way to turn the solution into integers

In [36]:
import numpy as np
import scipy.optimize as optimize

In [43]:
node_capacities = [10,5,7]
transport_capacities = [4,1,1]
demands = [[2,4,5,3],[0,1,1,1],[0,3,2,0]]
movements_orig = np.array(
                  [[[0,0,0],
                   [1,0,0],
                   [1,0,0]],
                  [[0,0,0],
                   [1,0,0],
                   [1,0,0]],
                  [[0,0,0],
                   [1,0,0],
                   [1,0,0]],
                  [[0,0,0],
                   [0,0,0],
                   [0,0,0]]
                 ])
prob_death_transport = 0

In [37]:
movements = movements_orig.ravel()
num_days = len(demands[0])
num_nodes = len(node_capacities)

In [38]:
def calculate_outgoing(array,day,node):
    return sum(array[day][node])


def calculate_incoming(array,day,node):
    total_outgoing = 0
    for n in range(num_nodes):
        total_outgoing = total_outgoing + array[day][n][node]
    return total_outgoing


def demand_day_node_raw(node_capacity, demand_day, left_day, received_day, demand_previous_days, left_previous_days, received_previous_days):
    return demand_day - left_day + received_day+ min(demand_previous_days-left_previous_days+received_previous_days, node_capacity)


def demand_day_node(movements,day,node):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    total_received_patients = 0
    total_outbound_patients = 0
    accumulated_demand = 0
    for d in range(day):
        total_received_patients = total_received_patients + calculate_incoming(movements_res,d,node)
        total_outbound_patients = total_outbound_patients + calculate_outgoing(movements_res,d,node)
    accumulated_demand = sum(demands[node][:day])
    
    demand = max(0, demand_day_node_raw(node_capacities[node],demands[node][day], 
                                        calculate_outgoing(movements_res,day,node), calculate_incoming(movements_res,day,node),
                                        accumulated_demand, total_outbound_patients, total_received_patients                
                ))
    return demand


def calc_total_deaths(movements):
    total = 0
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    for node in range(num_nodes):
        for day in range(num_days):
            deaths_not_attended = max(0, demand_day_node(movements,day,node) - node_capacities[node])
            deaths_transport = prob_death_transport*movements_res[day].sum()
            total = total + deaths_not_attended + deaths_transport
#             print(f'Node: {node}, Day: {day}, Deaths: {deaths_not_attended}, Total: {total}')
    return total


def outgoing_list(movements):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    outgoing_list = list()
    for day in range(num_days):
        for node in range(num_nodes):
            outgoing_list.append(calculate_outgoing(movements_res,day,node))
    return outgoing_list


def generate_bounds():
    outgoing_list = list()
    for day in range(num_days):
        for outgoing_node in range(num_nodes):
            for incoming_node in range(num_nodes):
                outgoing_list.append((0,transport_capacities[outgoing_node]))
    return outgoing_list

In [39]:
def cons1(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[0] - calculate_outgoing(movements_res,0,0)

def cons2(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[1] - calculate_outgoing(movements_res,0,1)

def cons3(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[2] - calculate_outgoing(movements_res,0,2)

def cons4(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[0] - calculate_outgoing(movements_res,1,0)
def cons5(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[1] - calculate_outgoing(movements_res,1,1)
def cons6(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[2] - calculate_outgoing(movements_res,1,2)
def cons7(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[0] - calculate_outgoing(movements_res,2,0)
def cons8(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[1] - calculate_outgoing(movements_res,2,1)
def cons9(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[2] - calculate_outgoing(movements_res,2,2)
def cons10(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[0] - calculate_outgoing(movements_res,3,0)
def cons11(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[1] - calculate_outgoing(movements_res,3,1)
def cons12(x):
    movements_res = np.reshape(x,(num_days,num_nodes,num_nodes))
    return transport_capacities[2] - calculate_outgoing(movements_res,3,2)

In [40]:
cons = ({'type': 'ineq', 'fun': cons1},
         {'type': 'ineq', 'fun': cons2},
         {'type': 'ineq', 'fun': cons3},
         {'type': 'ineq', 'fun': cons4},
         {'type': 'ineq', 'fun': cons5},
         {'type': 'ineq', 'fun': cons6},
         {'type': 'ineq', 'fun': cons7},
         {'type': 'ineq', 'fun': cons8},
         {'type': 'ineq', 'fun': cons9},
         {'type': 'ineq', 'fun': cons10},
         {'type': 'ineq', 'fun': cons11},
         {'type': 'ineq', 'fun': cons12})

In [41]:
initial_guess = movements.copy()

In [44]:
bounds_movement = generate_bounds()
result = optimize.minimize(calc_total_deaths, initial_guess, method='SLSQP', constraints=cons, bounds=bounds_movement)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [30]:
result.x

array([2.52435490e-28, 9.99511719e-01, 5.22848715e-20, 0.00000000e+00,
       0.00000000e+00, 9.99511719e-01, 1.59472155e-19, 0.00000000e+00,
       0.00000000e+00, 9.99511719e-01, 2.14994950e-19, 0.00000000e+00,
       0.00000000e+00, 9.99511719e-01, 0.00000000e+00, 0.00000000e+00])

In [31]:
np.reshape(np.rint(result.x),(num_days,num_nodes,num_nodes))

array([[[0., 1.],
        [0., 0.]],

       [[0., 1.],
        [0., 0.]],

       [[0., 1.],
        [0., 0.]],

       [[0., 1.],
        [0., 0.]]])

In [32]:
calc_total_deaths(np.rint(result.x))

2.0

In [33]:
calc_total_deaths(movements)

4

In [34]:
for node in range(num_nodes):
    for day in range(num_days):
        print(f'Node: {node}, Day: {day}, Demand: {demand_day_node(np.rint(result.x),day,node)}')

Node: 0, Day: 0, Demand: 1.0
Node: 0, Day: 1, Demand: 4.0
Node: 0, Day: 2, Demand: 8.0
Node: 0, Day: 3, Demand: 10.0
Node: 1, Day: 0, Demand: 1.0
Node: 1, Day: 1, Demand: 3.0
Node: 1, Day: 2, Demand: 5.0
Node: 1, Day: 3, Demand: 7.0


In [35]:
demand_day_node(np.rint(result.x),3,2)

IndexError: index 2 is out of bounds for axis 0 with size 2